In [ ]:
!pip install cirq --quiet
import cirq

from random import choices, getrandbits

In [ ]:
def alice_phase(num_bits, encode_gates, basis_gates, qubits):
  """PHASE 1 - ALICE SENDS"""

  # Step #1
  # Alice randomly picks bits
  alice_key = choices([0, 1], k = num_bits)

  # print('Alice\'s initial key: ', alice_key)

  # Step #2
  # Alice randomly picks bases
  alice_bases = choices(['Z', 'X'], k = num_bits)

  # print('\nAlice\'s randomly chosen bases: ', alice_bases)

  # Step #3
  # Alice Converts bits to qubits
  alice_circuit = cirq.Circuit()

  for bit in range(num_bits):

    encode_value = alice_key[bit]
    encode_gate = encode_gates[encode_value]

    basis_value = alice_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    alice_circuit.append(basis_gate(qubit))
    alice_circuit.append(encode_gate(qubit))
    # alice_circuit.append(basis_gate(qubit))

  # print('\nAlice\'s Phase 1 circuit:\n', alice_circuit)
  return alice_circuit, alice_key, alice_bases

In [ ]:
def bob_phase(num_bits, basis_gates, qubits, alice_circuit):
  """PHASE 2 - BOB RECEIVES"""
  # Step #4
  # Alice sends qubits to Bob

  # Step #5
  # Bob randomly pics bases
  bob_bases = choices(['Z', 'X'], k = num_bits)
  # print('Bob\'s randomly chosen bases: ', bob_bases)

  bob_circuit = cirq.Circuit()

  for bit in range(num_bits):

    basis_value = bob_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    bob_circuit.append(basis_gate(qubit))


  # Step #6
  # Bob measures qubits
  bob_circuit.append(cirq.measure(qubits, key = 'bob key'))
  # print('\nBob\'s Phase 2 circuit:\n', bob_circuit)


  # Step #7
  # Bob converts qubits back to bits
  bb84_circuit = alice_circuit + bob_circuit

  sim = cirq.Simulator()
  results = sim.run(bb84_circuit)
  bob_key = results.measurements['bob key'][0]

  # print('\nBob\'s initial key: ', list(bob_key))
  return list(bob_key), bob_bases

In [ ]:
def alice_bob_compare(num_bits, alice_bases, bob_bases, alice_key, bob_key):
  """PHASE 3 - ALICE & BOB COMPARE"""

  # Step #8
  # Alice and Bob compare bases and bits
  final_alice_key = []
  final_bob_key = []

  for bit in range(num_bits):
    if alice_bases[bit] == bob_bases[bit]:
      final_alice_key.append(alice_key[bit])
      final_bob_key.append(bob_key[bit])

  # print('\nAlice\'s key: ', final_alice_key)
  # print('Bob\'s key: ', final_bob_key)
  if not final_alice_key:
    return -1

  # Step #9
  # Alice and Bob create their key
  if final_alice_key[0] == final_bob_key[0]:
    final_alice_key = final_alice_key[1:]
    final_bob_key = final_bob_key[1:]

    if final_alice_key != final_bob_key:
      return -1
    else:
      return final_bob_key

  else:
    return -1

In [ ]:
def generate_key(num_bits=5):
  """Returns final key that bob and alice can use,
    If the keys at the end of the compare function are different then it returns -1
    If the keys first value are not the same then return -1 meaning that eve intercepted

    Uses BB84 implementation
  """
  encode_gates = {0: cirq.I, 1: cirq.X}
  basis_gates = {'Z': cirq.I, 'X': cirq.H}

  qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

  alice_circuit, alice_key, alice_bases = alice_phase(num_bits, encode_gates, basis_gates, qubits)
  # print("Alice key: ", alice_key)
  # print("ALice base:", alice_bases)

  bob_key, bob_bases = bob_phase(num_bits, basis_gates, qubits, alice_circuit)
  # print("Bob key:   ", bob_key)
  # print("Bob base:  ", bob_bases)

  final_key = alice_bob_compare(num_bits, alice_bases, bob_bases, alice_key, bob_key)

  return final_key

In [ ]:
def encrypt(string, key):
  zipped = []
  length = len(key)
  for idx in range(len(string)):
    i = string[idx]
    j = key[idx % length]
    zipped.append((i, j))

  return "".join(chr(ord(i) ^ ord(j)) for (i, j) in zipped)

In [ ]:
def decrypt(ciphertext, key):
  return encrypt(ciphertext, key)

In [ ]:
key = generate_key(num_bits=20)
if key == -1:
  print("Key's were not the same or Eve intercepted")
else:
  key = ''.join(str(e) for e in key)
  print(key)

001000010001


In [ ]:
message = "This message was encrypted with the help of QKD!"

encrypted_message = encrypt(message, key)
print(encrypted_message)
decrypted_message = decrypt(encrypted_message, key)
print(decrypted_message)

dXXC]UBCQWTGPCU^RBI@EUTGYDXDXUXU]@_Va{t
This message was encrypted with the help of QKD!
